# [Assistants migration guide](https://platform.openai.com/docs/assistants/migration)

# Constants and Libraries

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv

if not load_dotenv("./../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()
print("Environment variables have been loaded ;-)")
    
QUESTION="How many flights do we have between my cat Mike born date and Easter 2021?"

Environment variables have been loaded ;-)


# Create a client to connecto Azure OpenAI service and deployment

In [2]:
from openai import AzureOpenAI

# Create the client
client = AzureOpenAI(
    # api_key        = os.getenv("AZURE_OPENAI_API_KEY"),  
    # api_version    = os.getenv("AZURE_OPENAI_API_VERSION"), # at least 2024-02-15-preview
    # azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)
print(f"client.base_url: {client.base_url}")

client.base_url: https://mmoaiswc-01.openai.azure.com/openai/


# Define custom functions

In [3]:
def get_flights(date_1:str, date_2:str) -> dict:
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)

def my_cat_born_date(cat_name:str) -> dict:
    """ Returns my cat's born date """
    import datetime, random, json
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)

print(f'get_flights("2025-01-15", "2025-02-15") --> {get_flights(date_1 = "2025-01-15", date_2 = "2025-02-15")}')
print(f'my_cat_born_date --> {my_cat_born_date(cat_name = "Mike")}')

get_flights("2025-01-15", "2025-02-15") --> {"flights": 31}
my_cat_born_date --> {"cat_born_date": "2015-04-24"}


# Wrap the functions into their tools list

In [4]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_flights",
      "description": "returns the number of flights between two dates",
      "parameters": {
        "type": "object",
        "properties": {
          "date_1": {
            "type": "string",
            "description": "the first date"
          },
          "date_2": {
            "type": "string",
            "description": "the second date"
          }
        },
        "required": [
          "date_1",
          "date_2"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "my_cat_born_date",
      "description": "returns my cat's born date",
      "parameters": {
        "type": "object",
        "properties": {
          "cat_name": {
            "type": "string",
            "description": "the cat's name"
          }            
        },
        "required": []
      }
    }
  }
]

# Create an Assistant

In [5]:
assistant = client.beta.assistants.create(
    name="Smart Assistant",
    description="You are a helpful AI assistant who helps answering questions",
    tools = tools,
    model = "gpt-4o" # gpt-4.1 still not supported by Assistants API's
)

# print(assistant.model_dump_json(indent=2))
print(f"Assistant <{assistant.name}> (id = {assistant.id}) as been created.")

Assistant <Smart Assistant> (id = asst_MLvLDFv7E4mdrhhQdHy0yJOP) as been created.


# Create a conversation
Note that `code_interpreter` and `file_search` are empty in this case, because `code_interpreter` was not created.

In [6]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_Pu9TemMlWZYuKj8dIZf6KaYG', created_at=1745509858, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


# Add a message to the conversation

In [7]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=QUESTION
)

thread_messages = client.beta.threads.messages.list(thread.id)

print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_pbTtxugjPZxaE8vu9iCJRBid",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "How many flights do we have between my cat Mike born date and Easter 2021?"
          },
          "type": "text"
        }
      ],
      "created_at": 1745509858,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_Pu9TemMlWZYuKj8dIZf6KaYG"
    }
  ],
  "has_more": false,
  "object": "list",
  "first_id": "msg_pbTtxugjPZxaE8vu9iCJRBid",
  "last_id": "msg_pbTtxugjPZxaE8vu9iCJRBid"
}


# Create a Run and wait for its status to update

In [8]:
import time, json

run            = client.beta.threads.runs.create(
  thread_id    = thread.id,
  assistant_id = assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

while client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status in ["queued", "in_progress"]:
    print(f"Run status: {run.status}")
    time.sleep(5)

print(f"Final run status: {client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status}")

time.sleep(5)

json.loads(client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).to_json())

Run status: queued
Final run status: requires_action


{'id': 'run_rZ0hQqvuZLRUNOcTChp0Ujk6',
 'assistant_id': 'asst_MLvLDFv7E4mdrhhQdHy0yJOP',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1745509859,
 'expires_at': 1745545859,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': None,
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_1FBYyo7KeAiNDHPR3JMsrrIo',
     'function': {'arguments': '{"cat_name":"Mike"}',
      'name': 'my_cat_born_date'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'response_format': 'auto',
 'started_at': 1745509859,
 'status': 'requires_action',
 'thread_id': 'thread_Pu9TemMlWZYuKj8dIZf6KaYG',
 'tool_choice': 'auto',
 'tools': [{'function': {'name': 'get_flights',
    'description': 'returns the number of flights between two dates',
    'parameters': {'type': 'object',
  

# If `status` == `requires_action`, this cells runs the function(s) required

In [11]:
run_json = json.loads(client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).to_json())
if run_json["status"]=="requires_action":
    tool_outputs = []
    for tc in run_json['required_action']['submit_tool_outputs']['tool_calls']:
        tc_id = tc['id']
        tc_function_name = tc['function']['name']
        tc_args = tc['function']['arguments']
        tool_output = eval(tc_function_name)(**json.loads(tc_args))
        print(f"Calling {tc_function_name}({tc_args}) --> {tool_output}")
        tool_outputs.append({"tool_call_id": tc_id, "output": tool_output})
    print(f"tool_outputs: {tool_outputs}")
    
elif run_json["status"]=="queued":
    print("Hey, it's too early to run this cell, since the run status is 'queued'. Please wait a few seconds and try again.")

else:
    print(f'No more actions required. Run status: {run_json["status"]}')

Calling get_flights({"date_1":"2015-04-24","date_2":"2021-04-04"}) --> {"flights": 2172}
tool_outputs: [{'tool_call_id': 'call_IRlxsd5IpyIURA0Lh6P6Nsh9', 'output': '{"flights": 2172}'}]


# Pass result(s) of the executed function(s) back to the model via `tool_outputs`

# STOP HERE RUNNING THIS NOTEBOOK IN UNATTENDED MODE

In [12]:
if client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status == "completed":
    print("The run status is completed, you can move on and retrieve the final result ;-)")

else:
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        tool_outputs=tool_outputs,
        run_id=run.id
    )
    
    while client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status in ["queued", "in_progress"]:
        print(f"Run status: {run.status}")
        time.sleep(5)
    
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status
    
    print(f"Final run status: {run_status}")
    
    if run_status == "requires_action":
        print("Please run again the required function(s) in the above cell and then the current cell again, until the run is completed.")
    elif run_status == "completed":
        print("The run status is completed, you can move on and retrieve the final result ;-)")
    else:
        print(f"Please check why we have this run <{run_status}> status")

Run status: queued
Final run status: completed
The run status is completed, you can move on and retrieve the final result ;-)


# Extract the final result from the run
## Run the next cells **ONLY IF** `run.status = completed`

In [13]:
if client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status != "completed":
    print(f"Hey, it's too early to run this cell, since the run status is '{run_json['status']}'!")
    
else:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(json.loads(messages.to_json())["data"][0]["content"][0]["text"]["value"])

There are 2,172 flights between your cat Mike's birth date (April 24, 2015) and Easter 2021 (April 4, 2021).


## START teardown for the current assistant

In [14]:
for message in client.beta.threads.messages.list(thread.id):
    print (f"Deleting message id = <{message.id}>... of thread <{thread.id}>...")
    client.beta.threads.messages.delete(message_id=message.id, thread_id=thread.id)

print(f"\nDeleting thread {thread.id}...")
client.beta.threads.delete(thread_id=thread.id)

print(f"\nDeleting assistant {assistant.id} ({assistant.name})...")
client.beta.assistants.delete(assistant.id)

Deleting message id = <msg_pwiHcL50GZ5tuciRMP3ND5ng>... of thread <thread_Pu9TemMlWZYuKj8dIZf6KaYG>...
Deleting message id = <msg_pbTtxugjPZxaE8vu9iCJRBid>... of thread <thread_Pu9TemMlWZYuKj8dIZf6KaYG>...

Deleting thread thread_Pu9TemMlWZYuKj8dIZf6KaYG...

Deleting assistant asst_MLvLDFv7E4mdrhhQdHy0yJOP (Smart Assistant)...


AssistantDeleted(id='asst_MLvLDFv7E4mdrhhQdHy0yJOP', deleted=True, object='assistant.deleted')

# Final implementation in a single cell

In [15]:
import time, json

# Create the client
client = AzureOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key        = os.environ["AZURE_OPENAI_API_KEY"],
    api_version    = os.environ["OPENAI_API_VERSION"],
)

# Create the assistant
assistant = client.beta.assistants.create(
    name="Smart Assistant",
    description="You are a helpful AI assistant who helps answering questions",
    tools = tools,
    model = "gpt-4o" # gpt-4.1 still not supported by Assistants API's
)

# Create a thread
thread = client.beta.threads.create()

print (f"Question: {QUESTION}")

message       = client.beta.threads.messages.create(
    thread_id = thread.id,
    role      = "user",
    content   = QUESTION
)

run            = client.beta.threads.runs.create(
  thread_id    = thread.id,
  assistant_id = assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

start_time = time.time()
status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print( f"Run status: {status} after {int((time.time() - start_time) // 60)} minutes {int((time.time() - start_time) % 60)} seconds")
    
    if status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        for tc in tool_calls:
            function_to_call = tc.function.name
            function_args    = tc.function.arguments
            tool_output      = eval(function_to_call)(**json.loads(function_args))
            tool_outputs.append({"tool_call_id": tc.id, "output": tool_output})

            print(f"Calling {tc.function.name}(**{tc.function.arguments}) --> {tool_output}")

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

    time.sleep(5)
    status = run.status
        
print( f"\nRun status: {status} after {int((time.time() - start_time) // 60)} minutes {int((time.time() - start_time) % 60)} seconds")
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(json.loads(messages.to_json())["data"][0]["content"][0]["text"]["value"])

# Teardown
for message in client.beta.threads.messages.list(thread.id):
    print (f"Deleting message id = <{message.id}>... of thread <{thread.id}>...")
    client.beta.threads.messages.delete(message_id=message.id, thread_id=thread.id)

print(f"\nDeleting thread {thread.id}...")
client.beta.threads.delete(thread_id=thread.id)

print(f"\nDeleting assistant {assistant.id} ({assistant.name})...")
client.beta.assistants.delete(assistant.id)

Question: How many flights do we have between my cat Mike born date and Easter 2021?
Run status: queued after 0 minutes 0 seconds
Run status: in_progress after 0 minutes 5 seconds
Run status: requires_action after 0 minutes 11 seconds
Calling my_cat_born_date(**{"cat_name":"Mike"}) --> {"cat_born_date": "2015-04-24"}
Run status: queued after 0 minutes 16 seconds
Run status: requires_action after 0 minutes 22 seconds
Calling get_flights(**{"date_1":"2015-04-24","date_2":"2021-04-04"}) --> {"flights": 2172}
Run status: queued after 0 minutes 28 seconds

Run status: completed after 0 minutes 33 seconds
There are 2172 flights between your cat Mike's birth date, April 24, 2015, and Easter 2021 on April 4, 2021.
Deleting message id = <msg_KURBHxt6UG5YBmufj8yqvX32>... of thread <thread_q6E0e6k411uIU5GOMKU4oLW0>...
Deleting message id = <msg_HONxgVZCLB3twbKTFT3iIyDh>... of thread <thread_q6E0e6k411uIU5GOMKU4oLW0>...

Deleting thread thread_q6E0e6k411uIU5GOMKU4oLW0...

Deleting assistant asst_D

AssistantDeleted(id='asst_DwiIYVVJNqgllRUh7BiGaAQS', deleted=True, object='assistant.deleted')

# Teardown for *all** assistants and messages [(but **NOT** threads!)](https://learn.microsoft.com/en-us/answers/questions/2153170/assistants-api-where-and-how-long-are-entities-sav)
When creating a new thread with the Assistants API, thereby creating a stateful entity of: Threads, messages, where is this data stored, can I access the resource that stores these? Or is the resource managed entirely by Microsoft, and inaccessible to me?
When you create a new thread with the Assistants API, the data (threads, messages, etc.) is stored in a secure, Microsoft-managed storage account. This storage is logically separated to ensure data security. As a user, you do not have direct access to the underlying storage resources. Instead, you interact with the data through the API endpoints provided by Microsoft.

If a thread with messages is created via the API, and the ID is lost, is there then no route to access and delete this thread? As in, would there be a way to somehow fetch all threads related to a specific OpenAI Azure resource?
If you lose the thread ID, there is no direct way to retrieve or delete the thread through the API. The Assistants API does not currently provide a method to list all threads associated with a specific OpenAI Azure resource. Therefore, it's crucial to manage and store thread IDs securely within your application to ensure you can access and manage them as needed.

Is stateful entities stored INDEFINITELY unless deleted? Or is the a time to live if not used?
All used data persists in this system unless you explicitly delete this data. Use the delete function with the thread ID of the thread you want to delete. Clearing the Run in the Assistants Playground does not delete threads, however deleting them using delete function will not list them in the thread page.

In [16]:
from datetime import datetime

assistants = client.beta.assistants.list(limit=100).data
i=0
while len(assistants) > 0:
    for assistant in assistants: 
        i=i+1
        print(f"Deleting assistant {i}: {assistant.id} ({assistant.name}) created at {datetime.fromtimestamp(assistant.created_at).strftime('%Y-%m-%d %H:%M:%S')}...")
        client.beta.assistants.delete(assistant.id)
    assistants = client.beta.assistants.list(limit=100).data

print (f"\n{i} assistants have been successfully deleted.")

Deleting assistant 1: asst_EWk2HN9w9zlSB63hhjZWe2wA (Smart Assistant) created at 2025-04-24 17:45:01...
Deleting assistant 2: asst_jLmuSXlNmyhUANjfLFGd5wCL (Smart Assistant) created at 2025-04-24 17:34:47...
Deleting assistant 3: asst_2y1tE87dizN05A32nSllSoMn (Smart Assistant) created at 2025-04-24 17:26:29...
Deleting assistant 4: asst_V6CyPnSEp3PWovpdsqqRvaCN (Smart Assistant) created at 2025-04-24 17:18:27...
Deleting assistant 5: asst_hAV9QTeJaeEhe6nBSdZ5kNfS (Smart Assistant) created at 2025-04-24 17:15:31...
Deleting assistant 6: asst_abmn7NbPyWZzM1N7aCCLANVd (Smart Assistant) created at 2025-04-24 17:12:40...
Deleting assistant 7: asst_4HdthzdGQDJBT3sX8IUudNEg (Smart Assistant) created at 2025-04-24 17:01:58...

7 assistants have been successfully deleted.
